# Data Analytical Report
Reporting the result of the analysis

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

roundTrip_profit_g = pd.read_csv('../data/temproary_data/roundTrip_profit.csv')
roundTrip_profit_g['round_trip_route_IATA'] = roundTrip_profit_g['round_trip_route_IATA'].apply(lambda x: eval(x))
roundTrip_profit_g['inbound_FL_DATE'] = pd.to_datetime(roundTrip_profit_g['inbound_FL_DATE'])
roundTrip_profit_g['outbound_FL_DATE'] = pd.to_datetime(roundTrip_profit_g['outbound_FL_DATE'])

roundTrip_fre_g = pd.read_csv('../data/temproary_data/round_trip_flights.csv')
roundTrip_fre_g['round_trip_route_IATA'] = roundTrip_fre_g['round_trip_route_IATA'].apply(lambda x: eval(x))

## Evaluate the stability of the plane ticket itself

- weather the revenue is stable
- weather the cost is stable
- weather the profit is stable

In [72]:
# scaler = StandardScaler()


# def cal_weekly_stats(df) -> pd.DataFrame:
#     if df.shape[0] < 14:
#         return [np.NaN, np.NaN, np.NaN]

#     # bin data by date
#     df = df.set_index('pruchse_date')
#     df = df.resample('1D').agg({'total_cost':'sum', 'total_revenue':'sum', 'profit':'sum'})

#     # calculate moving average of 7 days window
#     df = df.rolling(window=7).mean().reset_index()

#     # calculate the standard deviation of the moving average of cost, revenue and profit
#     return [df['total_cost'].std(), df['total_revenue'].std(), df['profit'].std()]



# new_roundTrip_profit = roundTrip_profit.assign(pruchse_date = np.minimum(roundTrip_profit['inbound_FL_DATE'], roundTrip_profit['outbound_FL_DATE']))
# new_roundTrip_profit['total_cost'] = scaler.fit_transform(roundTrip_profit[['total_cost']])
# new_roundTrip_profit['total_revenue'] = scaler.fit_transform(roundTrip_profit[['total_revenue']])
# new_roundTrip_profit['profit'] = scaler.fit_transform(roundTrip_profit[['profit']])

# profit_stability = new_roundTrip_profit.groupby('round_trip_route_IATA')[['pruchse_date','total_cost', 'total_revenue', 'profit']].apply(cal_weekly_stats).reset_index()

# profit_stability = profit_stability.assign(rolling_avg_cost = profit_stability[0].apply(lambda x: x[0]))
# profit_stability = profit_stability.assign(rolling_avg_cost_rank = profit_stability['rolling_avg_cost'].rank(ascending=True))
# profit_stability = profit_stability.assign(rolling_avg_revenue = profit_stability[0].apply(lambda x: x[1]))
# profit_stability = profit_stability.assign(rolling_avg_revenue_rank = profit_stability['rolling_avg_revenue'].rank(ascending=True))
# profit_stability = profit_stability.assign(rolling_avg_profit = profit_stability[0].apply(lambda x: x[2]))
# profit_stability = profit_stability.assign(rolling_avg_profit_rank = profit_stability['rolling_avg_profit'].rank(ascending=True))
# profit_stability.drop(columns=0, inplace=True)

In [2]:
new_roundTrip_profit = roundTrip_profit_g.assign(purchase_date = np.minimum(roundTrip_profit_g['inbound_FL_DATE'], roundTrip_profit_g['outbound_FL_DATE']))
new_roundTrip_profit = new_roundTrip_profit.groupby(['round_trip_route_IATA', pd.Grouper(key = 'purchase_date', freq='100D')])['profit'].mean()

new_roundTrip_profit =  new_roundTrip_profit.reset_index().groupby('round_trip_route_IATA').mean().rename(columns={'profit':'weekly_avg_profit'})
new_roundTrip_profit['weekly_avg_rank'] = new_roundTrip_profit['weekly_avg_profit'].rank(ascending=False, method='min')
# new_roundTrip_profit = new_roundTrip_profit[['weekly_avg_rank', 'profit']]
new_roundTrip_profit = new_roundTrip_profit.reset_index()

In [3]:
new_roundTrip_profit

,round_trip_route_IATA,weekly_avg_profit,weekly_avg_rank
0,"(ABE, ATL)",44051.677068,158.0
1,"(ABE, CLT)",39060.526414,276.0
2,"(ABE, DTW)",13615.185832,1418.0
3,"(ABE, FLL)",-4426.697750,2488.0
4,"(ABE, ORD)",35733.344067,364.0
...,...,...,...
2770,"(STL, TPA)",7939.627630,1834.0
2771,"(STL, TUL)",15727.713507,1289.0
2772,"(SYR, TPA)",-4749.308920,2500.0
2773,"(TPA, TTN)",-22160.059612,2761.0


## Aggregrating all the factors and assign ranks to each factors for round-trip route
consider factors:
- flight frequency
- flight profit (total profit - total cost)
- TODO: weekly profit

In [5]:
# calculate the ranking of the profit
roundTrip_profit = (roundTrip_profit_g.groupby('round_trip_route_IATA')['profit']
                 .sum()
                 .reset_index()
                 .rename(columns={'profit':'round_trip_profit'}))
roundTrip_profit = (roundTrip_profit.assign(round_trip_profit_ranking = roundTrip_profit['round_trip_profit']
                                    .rank(ascending = False, method = 'min')))


# calculate the ranking of the frequency
roundTrip_fre = (roundTrip_fre_g.groupby('round_trip_route_IATA')['outbound_OCCUPANCY_RATE']
                 .count().sort_values(ascending=False)
                 .reset_index()
                 .rename(columns={'outbound_OCCUPANCY_RATE':'round_trip_flights_count'}))
roundTrip_fre = (roundTrip_fre.assign(round_trip_flights_count_ranking = roundTrip_fre['round_trip_flights_count']
                              .rank(ascending = False, method = 'min')))


# calculate the ranking of the operation career
roundTrip_op = (roundTrip_profit_g
                .groupby('round_trip_route_IATA')
                .apply(lambda df: len(set(df['inbound_OP_CARRIER']).union(set(df['outbound_OP_CARRIER']))))
                .reset_index()
                .rename(columns={0:'round_trip_op_count'}))
# roundTrip_op = (roundTrip_op.assign(round_trip_op_count_ranking = roundTrip_op['round_trip_op_count']
#                               .rank(ascending = True, method = 'min')))


# # merge the ranks
round_trip_info = roundTrip_profit.merge(roundTrip_fre, on='round_trip_route_IATA', how='inner')

round_trip_info = round_trip_info.merge(new_roundTrip_profit, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_op, on='round_trip_route_IATA', how='left')

# # # aggregate the ranking
round_trip_info = round_trip_info.assign(avg_ranking = ((round_trip_info['round_trip_flights_count_ranking'] * 0.2 + round_trip_info['round_trip_profit_ranking'] * 0.8) + round_trip_info['weekly_avg_rank'] * 0)/3)
round_trip_info.sort_values(by='avg_ranking', ascending=True).head(20)


,round_trip_route_IATA,round_trip_profit,round_trip_profit_ranking,round_trip_flights_count,round_trip_flights_count_ranking,weekly_avg_profit,weekly_avg_rank,round_trip_op_count,avg_ranking
1978,"(JFK, LAX)",1.065975e+08,1.0,3140,4.0,33948.249548,422.0,4,0.533333
1109,"(DCA, ORD)",4.670428e+07,8.0,1843,22.0,25341.444184,771.0,6,3.600000
2004,"(JFK, SFO)",4.663502e+07,9.0,1842,23.0,25317.600233,774.0,4,3.933333
128,"(ATL, CLT)",4.719618e+07,7.0,1534,43.0,30766.742344,535.0,3,4.733333
1093,"(DCA, LGA)",4.070571e+07,12.0,1674,35.0,24316.435876,833.0,2,5.533333
1555,"(EWR, SFO)",5.832132e+07,3.0,1202,94.0,48520.229339,100.0,2,7.066667
1283,"(DFW, IAH)",3.901894e+07,14.0,1477,54.0,26417.700084,721.0,7,7.333333
2392,"(MSP, ORD)",3.299944e+07,25.0,1717,30.0,19219.243014,1090.0,9,8.666667
182,"(ATL, LGA)",3.012466e+07,37.0,2292,10.0,13143.393514,1449.0,5,10.533333
2123,"(LAX, SAN)",3.051876e+07,33.0,1410,59.0,21644.513495,961.0,4,12.733333
